<a href="https://colab.research.google.com/github/marekhudec/MKA-MLF/blob/main/Final_Projekt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TRÉNOVACÍ MODEL
# Načítanie knižníc
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from PIL import Image
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

# Výber zariadenia CPU alebo GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# definícia modelu CNN
class CustomCNN(nn.Module):
    def __init__(self, num_classes, conv_channels=[16, 32, 64], kernel_sizes=[3, 3, 3], fc_sizes=[512]):
        super(CustomCNN, self).__init__()
        self.layers = nn.ModuleList()

        # Vytvorenie konvolučných vrstiev
        in_channels = 3
        for out_channels, kernel_size in zip(conv_channels, kernel_sizes):
            self.layers.append(
                nn.Sequential(
                    nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=1, padding=kernel_size//2),
                    nn.ReLU(),
                    nn.MaxPool2d(kernel_size=2, stride=2)
                )
            )
            in_channels = out_channels

        self.flatten = nn.Flatten()

        # Vytvorenie flatten a fully-connected vrstiev
        linear_layers = []
        input_features = conv_channels[-1] * (64 // 2**len(conv_channels))**2
        for output_features in fc_sizes:
            linear_layers.append(nn.Linear(input_features, output_features))
            linear_layers.append(nn.ReLU()) # Aktivačná funkcia
            input_features = output_features
        linear_layers.append(nn.Linear(input_features, num_classes))
        self.fc = nn.Sequential(*linear_layers)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x


In [ ]:
# definícia a načítanie datasetu
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = f"{self.root_dir}/img_{idx + 1}.png"
        image = Image.open(img_name).convert('RGB') # konverzia na RGB
        label = int(self.data_frame.iloc[idx, 1])
        if self.transform:
            image = self.transform(image)
        return image, label

# načítanie štítkov
labels_csv = r'C:\Users\Asus\Desktop\CNN\y_train.csv'

# načítanie priečinka s obrázkami + data augumentation
image_root = r'C:\Users\Asus\Desktop\CNN\train_data_unlabeled'
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
# rozdelenie dát na trénovacie a validačné
dataset = CustomDataset(csv_file=labels_csv, root_dir=image_root, transform=transform)
train_size = 0.9
train_indices, val_indices = train_test_split(list(range(len(dataset))), train_size=train_size, random_state=42)
train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)



In [ ]:
# definícia variabilných hyperparametrov
hyperparameters = {
    'lr': [0.01],
    'batch_size': [16, 32, 64],
    'optimizer': ['adam', 'sgd'],
    'num_epochs': [4, 5, 6],
    'conv_channels': [ (32, 64, 128)],
    'kernel_sizes': [(3, 3, 3), (5, 3, 3)],
    'fc_sizes': [(512,), (256, 128), (1024,)]
}

# Funkcia pre náhodný výber hyperparametrov
def sample_hyperparameters(hyperparameters):
    return {k: random.choice(v) for k, v in hyperparameters.items()}

# definícia optimizéru Adama/SGD
def get_optimizer(optimizer_name, parameters, lr):
    if optimizer_name == 'adam':
        return optim.Adam(parameters, lr=lr)
    elif optimizer_name == 'sgd':
        return optim.SGD(parameters, lr=lr, momentum=0.9)

# Initialize plotting variables
best_metrics = {
    'train_loss': [],
    'val_loss': [],
    'accuracy': []
}



In [ ]:
# implementácia trénovacej slučky s použitím Random search algoritmu
num_trials = 8
best_accuracy = 0.87
best_model = None
best_params = None

for trial in range(num_trials):
    params = sample_hyperparameters(hyperparameters)
    print(f"Trial {trial+1}: Testing with parameters: {params}")

    model = CustomCNN(
        num_classes=4,
        conv_channels=params['conv_channels'],
        kernel_sizes=params['kernel_sizes'],
        fc_sizes=params['fc_sizes']
    ).to(device)
  # Loaderi pre data , nastavenie batch_size
    train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=params['batch_size'], shuffle=False)

    optimizer = get_optimizer(params['optimizer'], model.parameters(), params['lr'])
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    # Training and validation loops
    epoch_train_loss = []
    epoch_val_loss = []
    epoch_accuracy = []

    for epoch in range(params['num_epochs']): # trénovací cyklus
        model.train()
        total_train_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item() * images.size(0)
        scheduler.step()

        model.eval()
        total_val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():# validačný cyklus
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                total_val_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        train_loss = total_train_loss / len(train_loader.dataset)
        val_loss = total_val_loss / len(val_loader.dataset)
        accuracy = 100 * correct / total

        epoch_train_loss.append(train_loss)
        epoch_val_loss.append(val_loss)
        epoch_accuracy.append(accuracy)

        print(f"Epoch {epoch+1}/{params['num_epochs']}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%") # výpis priebežných výsledkov poča epoch

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
        best_params = params
        best_metrics['train_loss'] = epoch_train_loss
        best_metrics['val_loss'] = epoch_val_loss
        best_metrics['accuracy'] = epoch_accuracy



In [ ]:
# vykreslenie grafov
epochs = range(1, len(best_metrics['train_loss']) + 1)
plt.figure(figsize=(12, 8))
plt.subplot(2, 1, 1)
plt.plot(epochs, best_metrics['train_loss'], label='Training Loss')
plt.plot(epochs, best_metrics['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, best_metrics['accuracy'], color='green', label='Accuracy')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.tight_layout()
plt.show()

# uloženie najlepšieho výsledku modelu
torch.save(best_model.state_dict(), 'best_custom_cnn_model.pth')
print(f"Best model saved with accuracy {best_accuracy:.2f}% and parameters {best_params}")


In [ ]:
#TESTOVACÍ MODEL
# načítanie knižníc
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import pandas as pd
import os


# Testovací model s hodnotami najlepšieho výsledku trénovacieho modelu
class CustomCNN(nn.Module):
    def __init__(self, num_classes, conv_channels=[32, 64, 128], kernel_sizes=[5, 3, 3], fc_sizes=[512]):
        super(CustomCNN, self).__init__()
        self.layers = nn.ModuleList()
        in_channels = 3
        for out_channels, kernel_size in zip(conv_channels, kernel_sizes):
            self.layers.append(
                nn.Sequential(
                    nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=1,
                              padding=kernel_size // 2),
                    nn.ReLU(),
                    nn.MaxPool2d(kernel_size=2, stride=2)
                )
            )
            in_channels = out_channels

        self.flatten = nn.Flatten()
        linear_layers = []
        input_features = conv_channels[-1] * (64 // 2 ** len(conv_channels)) ** 2
        for output_features in fc_sizes:
            linear_layers.append(nn.Linear(input_features, output_features))
            linear_layers.append(nn.ReLU())
            input_features = output_features
        linear_layers.append(nn.Linear(input_features, num_classes))
        self.fc = nn.Sequential(*linear_layers)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x


# Načítanie uloženého trénovacieho modelu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CustomCNN(num_classes=4).to(device)
model_path = 'best_custom_cnn_model.pth'
model.load_state_dict(torch.load(model_path))
model.eval()

# data augumentation
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Načítanie testovacích dát
image_dir = r'C:\Users\Asus\Desktop\CNN\test_data_unlabeled'

results = []
image_id = 0

# testovanie obrázkov z testovacieho datasetu
for filename in os.listdir(image_dir):
    if filename.endswith(".png"):
        image_path = os.path.join(image_dir, filename)
        image = Image.open(image_path).convert('RGB')
        image_tensor = transform(image).unsqueeze(0).to(device)  # Add batch dimension and move to device

        # Perform inference
        with torch.no_grad():
            output = model(image_tensor)
            predicted_class = torch.argmax(output, dim=1).item()

        # Store results
        results.append({
            "id": image_id,
            "target": predicted_class
        })
        image_id += 1

# Konverzia výsledkov do CSV
df = pd.DataFrame(results)
df.to_csv('output_predictions.csv', index=False, columns=['id', 'target'])

print("Finished processing and saved results to 'output_predictions.csv'")
